In [262]:
# import pandas lib as pd
import pandas as pd
import numpy as np
# read by default 1st sheet of an excel file
df= pd.read_excel('datatest.xlsx',header=0, sheet_name="Style1")
df.insert(2,'sewing',0,True)
df.insert(3,'wating',df['headcount'],True)
df

,sequence,operation,sewing,wating,SAM,headcount,WIP
0,1,Bind Panel,0,4,3.3,4,0
1,2,Over Edge,0,4,3.6,4,0
2,3,Seam Panel,0,20,15.0,20,18
3,4,Seam Crotch,0,7,7.4,7,10
4,5,Hem Legs,0,8,8.6,8,6
5,6,Sew Band,0,9,9.2,9,0


In [263]:
# import pandas lib as pd
import pandas as pd
# read by default 1st sheet of an excel file
df2= pd.read_excel('datatest.xlsx',header=0, sheet_name="Style2")
df2.insert(2,'sewing',0,True)
df2.insert(3,'wating',df2['headcount'],True)
df2

,sequence,operation,sewing,wating,SAM,headcount,WIP
0,1,Bind Panel,0,4,3.3,4,7
1,2,Over Edge,0,4,3.6,4,25
2,3,Attach Sling Rubber,0,7,6.0,7,10
3,4,Sew Center,0,8,8.0,8,14
4,5,Topstitch,0,6,4.0,6,8
5,6,Seam Panel,0,20,15.0,20,18
6,7,Seam Crotch,0,7,7.4,7,10
7,8,Hem Legs,0,8,8.6,8,6
8,9,Sew Band,0,9,9.2,9,0


In [264]:
# import pandas lib as pd
import pandas as pd
# read by default 1st sheet of an excel file
df3= pd.read_excel('datatest.xlsx',header=0, sheet_name="OPERATION_CONFIG")
# df3=df3.drop([0])
df3

,style,operation,next operation
0,Style 1,Bind Panel,Seam Panel
1,Style 1,Over Edge,Seam Panel
2,Style 2,Bind Panel,Sew Center
3,Style 2,Over Edge,Sew Center
4,Style 2,Sew Center,Topstitch
5,Style 2,Attach Sling Rubber,Topstitch


In [265]:
# import pandas lib as pd
import pandas as pd
# read by default 1st sheet of an excel file
df4= pd.read_excel('datatest.xlsx',header=0, sheet_name="NO_WIP_REQ")
# df4=df4.drop([1])
df4

,style,operation
0,Style 1,Bind Panel
1,Style 1,Over Edge
2,Style 2,Bind Panel
3,Style 2,Over Edge
4,Style 2,Attach Sling Rubber


In [281]:

def update_sequence(df_style,point_check):
    sequence=point_check[1]
    sew_collect=point_check[2]
    df_style.loc[sequence-1,'sewing']-=sew_collect
    df_style.loc[sequence-1,'wating']+=sew_collect
    df_style.loc[sequence-1,'WIP']+=sew_collect
    
def cal_time_check(df_style,time,num_seq,sew,list_point_check):
    output=[]
    point_time=time + df_style.loc[num_seq-1,'SAM']
    output.append(round(point_time,1))
    output.append(num_seq)
    output.append(sew)
    list_point_check.append(output)
    list_point_check.sort()
    return list_point_check
    
def sequence_first(df_style,ti_step,ti_line,num_seq,list_point_check):
    df_style.loc[num_seq-1,'wating']=0
    df_style.loc[num_seq-1,'sewing']=df_style.loc[num_seq-1,'headcount']
    if ti_step==0:
        cycle=ti_line//df_style.loc[num_seq-1,'SAM']
        for cyc in range(0,int(cycle)):
            cal_time_check(df_style,cyc*df_style.loc[num_seq-1,'SAM'],num_seq,df_style.loc[num_seq-1,'headcount'],list_point_check)

    
def cal_wip(df_style,num_seq,time,list_point_check,list_seq_first,list_num_seq_dep,list_seq_depend):
    name_seq=df_style.loc[num_seq-1,"operation"]
    ls_wip=[]
    if name_seq in list_seq_first:
        sequence_first(df_style,0.1,0.1,num_seq,list_point_check)
    elif name_seq in list_seq_depend:
        for item in list_num_seq_dep:
            try:
                num=item[0]
            except:
                break
            if num_seq == num :
                for i in item[1]:
                    ls_wip.append(df_style.loc[df_style["sequence"]==i,'WIP'])
                min_wip=np.array(ls_wip).min()
                if df_style.loc[num_seq-1,'wating']<=min_wip:
                    cal_time_check(df_style,time,num_seq,df_style.loc[num_seq-1,'wating'],list_point_check)
                    df_style.loc[num_seq-1,'sewing']=df_style.loc[num_seq-1,'headcount']
                    df_style.loc[num_seq-1,'wating']=0
                    for i in item[1]:
                        df_style.loc[i-1,'WIP']-=df_style.loc[num_seq-1,'wating']
                
                if df_style.loc[num_seq-1,'wating']>=min_wip and min_wip >0:
                    cal_time_check(df_style,time,num_seq,min_wip,list_point_check)
                    df_style.loc[num_seq-1,'sewing']+=min_wip
                    df_style.loc[num_seq-1,'wating']-=min_wip
                    for i in item[1]:
                        df_style.loc[i-1,'WIP']-=min_wip
    else:
        if df_style.loc[num_seq-1,'wating'] > 0 and df_style.loc[num_seq-2,'WIP'] > 0 and df_style.loc[num_seq-2,'WIP'] <= df_style.loc[num_seq-1,'wating']:
            cal_time_check(df_style,time,num_seq,df_style.loc[num_seq-2,'WIP'],list_point_check)
            df_style.loc[num_seq-1,'wating']-=df_style.loc[num_seq-2,'WIP']
            df_style.loc[num_seq-1,'sewing']+=df_style.loc[num_seq-2,'WIP']
            df_style.loc[num_seq-2,'WIP']=0

        if df_style.loc[num_seq-1,'wating']> 0 and df_style.loc[num_seq-2,'WIP']> df_style.loc[num_seq-1,'wating']:
            cal_time_check(df_style,time,num_seq,df_style.loc[num_seq-1,'wating'],list_point_check)
            df_style.loc[num_seq-2,'WIP']-=df_style.loc[num_seq-1,'wating']
            df_style.loc[num_seq-1,'wating']=0
            df_style.loc[num_seq-1,'sewing']=df_style.loc[num_seq-1,'headcount']


def get_sequence_first(df_no_wip_req, name_style):
    list_seq_first=df_no_wip_req[df_no_wip_req["style"]==name_style]["operation"].values
    return list_seq_first

def get_sequence_depend(df_config, name_style):
    list_seq_depend=[]
    list_seq_previous=[]
    df_=df_config[df_config["style"]==name_style]
    list_seq_depend=df_["next operation"].unique()    
    for i in list_seq_depend:
        a=df_[df_["next operation"]==i]["operation"].values
        list_seq_previous.append(a)
    return list_seq_depend, list_seq_previous

def check_sequence(df_style,list_seq_first,list_seq_depend,list_seq_previous,ti_line,num_seq,list_point_check,list_num_seq_dep):
    name_seq=df_style.loc[num_seq-1,"operation"]
    item_dep=[]
    if name_seq in list_seq_first:
        sequence_first(df_style,0,ti_line,num_seq,list_point_check)
    if name_seq in list_seq_depend:
        item_dep.append(num_seq)
        index=np.where(list_seq_depend==name_seq)[0][0]
        for i in list_seq_previous[index]:
            item_dep.append(df_style[df_style["operation"]==i]["sequence"].values)
        list_num_seq_dep.append(item_dep)
    
def main(df_style,df_config,df_no_wip_req,name_style,time):
    
    time_step=0
    time_line=time
    list_first= get_sequence_first(df_no_wip_req,name_style)
    list_depend,list_previous=get_sequence_depend(df_config,name_style)
    lis_num_dep=[]
    list_check=[]
    for num_seq in range(1,df_style.shape[0]+1):
        check_sequence(df_style,list_first,list_depend,list_previous,time_line,num_seq,list_check,lis_num_dep)
        cal_wip(df_style,num_seq,time_step,list_check,list_first,lis_num_dep,list_depend)
    pcheck=list_check.pop(0)
    time_step=pcheck[0]
    while time_step<= time_line:
        update_sequence(df_style,pcheck)
        for num_seq in range(1,df_style.shape[0]+1):
            cal_wip(df_style,num_seq,time_step,list_check,list_first,lis_num_dep,list_depend)
        try:
            pcheck=list_check.pop(0)
            time_step=pcheck[0]
        except:
            break
    return df_style

In [282]:
df= pd.read_excel('datatest.xlsx',header=0, sheet_name='Style1')
df.insert(2,'sewing',0,True)
df.insert(3,'wating',df['headcount'],True)
df_conf= pd.read_excel('datatest.xlsx',header=0, sheet_name='OPERATION_CONFIG')
df_nwr= pd.read_excel('datatest.xlsx',header=0, sheet_name='NO_WIP_REQ')


output=main(df,df_conf,df_nwr,'Style 1',30)
output

,sequence,operation,sewing,wating,SAM,headcount,WIP
0,1,Bind Panel,4,0,3.3,4,20
1,2,Over Edge,4,0,3.6,4,32
2,3,Seam Panel,20,0,15.0,20,2
3,4,Seam Crotch,7,0,7.4,7,3
4,5,Hem Legs,8,0,8.6,8,0
5,6,Sew Band,8,1,9.2,9,22
